### Quellen
* http://brandonrose.org/clustering IMDB top 100
* https://medium.com/@MSalnikov/text-clustering-with-k-means-and-tf-idf-f099bcf95183

# Transformation der Daten

In diesem Notebook sollen vorher verarbeitete Daten der Filmzusammenfassungen genutzt werden, um einen weiteren Mehrwehrt zu generieren. Das Ziel ist es, die Funktionalität eines konkreten Anwendungsfalls von Machine Learnining (ML) im Bereich der Filme zu realisieren.

### Problemstellung

Wir möchten uns in diesem Projekt letztendlich damit beschäftigen, eine Clusteringalgorithmus zu entwerfen, welcher uns eine Ähnlichkeitsanalyse von Filmen erlaubt. Dieses Thema interessiert und vor allem, da es ein konkreten Anwendungsfall für die Vorhersage von ähnlichen Filmen, gerade im Bereich des Onlinestreamings, darstellt. Zudem können wir uns mit einem aus der Vorlesung bekannten Clustering-Algorithmus beschäftigen und darauf aufbauend unser Wissen vertiefen.

### Anwendungsbeispiel

Die amerikanische Onlinestreaming-Plattform [Netflix](https://www.netflix.com/de-en/) bietet beispielsweise ein Feature, wobei weitere Vorschläge, basierend auf dem Konsum des Kunden, dargestellt werden. 

* TODO: BILD

Welche Datenbasis hierfür verwendet wird ist unklar. Es kommen generell mehrere Quellen in Frage:

* Bewertungen des Nutzers
* Streaming-Verlauf des Nutzers
* Filme des gleichen Genres
* Inhaltlich ähnliche Filme
* Filmneuheiten
* uvm.

Welche Quelle Netflix für ihr Feature nutzt bleibt unklar. Sehr wahrscheinlich handelt es sich dabei um eine Kombination vorherigen Quellen. 

### Idee

Wir möchten uns im Rahmen dieses Projekts mit einer Variante dieses Features beschäftigen, der Generierung von Vorschlägen basierend auf inhaltlicher Ähnlichkeit. Hierzu nutzen wir inhaltliche Zusammenfassungen der Filme in Textform, welche wir durch einen selbstgeschriebenen Crawler generiert haben.

### Module importieren

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import spacy
import numpy as np
from sklearn.model_selection import train_test_split

### Daten einlesen

In [2]:
PATH_TO_DATA = '../data/raw.json'
# JSON Daten in Dataframe lesen
data = pd.read_json(PATH_TO_DATA)

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
# Neues DataFrame initiieren
bag_of_words = pd.DataFrame(columns=['title', 'bag_of_words'])
# Iteriere über jeden Film in Dataframe
for index, movie in data.iterrows():
    # Tokenisieren von Dokument
    document = nlp(movie['synopsis'])
    # Kopiere alle Eigennamen
    entities = set([entity.text for entity in document.ents])
    lemmas = []
    for token in document:
        # Nimm lemmatisiertes Token falls kein Eigenname, Stoppwort und Satzzeichen
        if token.text not in entities and not token.is_stop and not token.is_punct:
            lemmas.append(token.lemma_)
    # Ersetze Zusammenfassung in Dataframe mit vorverarbeitetem Bag of Words
    data.loc[index, 'synopsis'] = ' '.join(lemmas)

Wir benennen ebenfalls unsere Spalte *synopsis* in *bag_of_words* um.

In [5]:
data.rename(columns={'synopsis':'bag_of_words'}, inplace=True)

Das DataFrame sieht nun folgendermaßen aus:

In [7]:
data

,title,bag_of_words
0,Godfellas,film open man drive car late night highway car...
1,Casino,Martin Scorsese film Casino follow life Rothst...
2,The Lord of the Rings: The Fellowship of the Ring,prologue speak show Dark Lord forge use conque...
3,The Hobbit: An Unexpected Journey,prologue preparation birthday party beginning ...


### Bag of Words in Datei schreiben

Damit wir die Vorverarbeitung der Daten nicht bei jedem Neustart des Trainingsprozesses durchführen müssen, schreiben wir eine neue Datei.

In [8]:
data.to_csv("../data/bag_of_words.csv")

### Aufteilen in Trainings- und Testdaten

In [9]:
X_train, X_test, y_train, y_test = train_test_split(list(data['bag_of_words']), data['title'], test_size=0.1, random_state=42)

TODO: Train test nutzen

### Features

* Word-Matrix
* Frequency
* Normalisieren

TODO: Vektorisieren etc. selbst machen?

In [10]:
# Matrix von Token mit Frequenz plus Normalisieren mittels "Inverse-document-frequency" (IDF)
vectorizer = TfidfVectorizer()
# Lernen des Vokabulars und IDF
X = vectorizer.fit_transform(data['bag_of_words'])

In [11]:
X.shape

(4, 1432)

In [2]:
X.toarray()

NameError: name 'X' is not defined

In [13]:
feature_names = vectorizer.get_feature_names()
feature_names

['000',
 '11th',
 '13',
 '14th',
 '1950',
 '1970',
 '1978',
 '1980',
 '35',
 '500',
 '60',
 'abandon',
 'abet',
 'able',
 'absent',
 'abuse',
 'abyss',
 'accept',
 'acceptance',
 'accidentally',
 'accompany',
 'accustomed',
 'ace',
 'act',
 'action',
 'actual',
 'adam',
 'addict',
 'address',
 'admit',
 'adopt',
 'adult',
 'advantage',
 'adventure',
 'affair',
 'affect',
 'afloat',
 'afraid',
 'agent',
 'aggravate',
 'agree',
 'aid',
 'aidan',
 'aide',
 'air',
 'airport',
 'alan',
 'albeit',
 'alcoholic',
 'alimony',
 'alive',
 'alliance',
 'allison',
 'allow',
 'ally',
 'alright',
 'alternately',
 'ambush',
 'ancient',
 'anduin',
 'andy',
 'angmar',
 'angrily',
 'angry',
 'animal',
 'anonymity',
 'answer',
 'apart',
 'apartment',
 'apologize',
 'apparently',
 'appear',
 'appearance',
 'area',
 'argue',
 'arkenstone',
 'arm',
 'armitage',
 'army',
 'arnor',
 'arouse',
 'arrest',
 'arrive',
 'artie',
 'arwen',
 'as',
 'ask',
 'asleep',
 'associate',
 'assure',
 'atop',
 'attack',
 'atte

### Clustering

* Um evaluieren zu koennen 1- len(data) - 1 Cluster ausprobieren, inertia abspeichern und in Graph visualisieren
* k waehlen mit Begruendung

In [14]:
# for k in range(len(data) - 1):
#     model = KMeans(n_clusters=k, init='k-means++', max_iter=1000)
#     model.fit(X)
    # Daten abspeichern (inertia, n_iter, ...)

In [15]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=1000)
model.fit(X)

KMeans(max_iter=1000, n_clusters=2)

### Ergebnisse

In [16]:
model.n_iter_

2

In [17]:
model.inertia_

1.4972744701501992

In [18]:
model.labels_

array([1, 1, 0, 0], dtype=int32)

### TODO: Ergebnisse/Visualisierung

* Cluster Feature untersuchen/interpretieren/visualieren

In [19]:
# Indizes von Tokens pro Cluster nach Relevanz sortieren
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# 10 wichtigsten Tokens von jeweiligem Cluster ausgeben
for i in range(true_k):
    print("\nCluster {}:\n{}".format(i, ', '.join(feature_names[ind] for ind in order_centroids[i, :15])))


Cluster 0:
dwarf, orcs, hobbit, goblin, leave, ring, mountain, doom, escape, sword, mount, destroy, forge, give, elf

Cluster 1:
casino, tell, family, money, boss, man, drug, start, get, give, police, ask, kill, murder, shoot


### Ähnlichkeiten innerhalb Cluster bestimmen

In [29]:
nlp = spacy.load("en_core_web_lg")
nlp(data['bag_of_words'][1]).similarity(nlp(data['bag_of_words'][2]))

0.9195162791347236

### Testen

In [21]:
# JSON Daten in Dataframe lesen
data_test = pd.read_json('../data/test.json')

tokens_test = {}

# Iteriere über jeden Film in Dataframe
for index, movie in data_test.iterrows():
    # Tokenisieren von Dokument
    document = nlp(movie['synopsis'])
    # Kopiere alle Eigennamen
    entities = set([entity.text for entity in document.ents])
    lemmas = []
    for token in document:
        # Nimm lemmatisiertes Token falls kein Eigenname, Stoppwort und Satzzeichen
        if token.text not in entities and not token.is_stop and not token.is_punct:
            lemmas.append(token.lemma_)
    # Bilde zusammengesetzten String von relevanten Token
    tokens_test[movie['title']] =  ' '.join(lemmas)

In [22]:
X_test = vectorizer.transform(tokens_test.values())

### Ergebnis Prediktion

In [23]:
model.predict(X_test)

array([1], dtype=int32)

### Fragen

* Wie benenne ich Cluster
* Wie kann ich evaluieren, wie viele Cluster sinnvoll?

Compute clustering algorithm (e.g., k-means clustering) for different values of k. For instance, by varying k from 1 to 10 clusters.
For each k, calculate the total within-cluster sum of square (wss).
Plot the curve of wss according to the number of clusters k.
The location of a bend (knee) in the plot is generally considered as an indicator of the appropriate number of clusters.